# Ülkeler baz alındığında Temel İçme Suyuna Sahip Olmanın, Çocukların Ölümü Üzerine Etkisi

İncelediğiniz kaggle not defteri sadece kodları içermektedir.

Analiz üzerine yorumlar Medium hikayemde bulunmaktadır.

Medium Hikayem : https://16savasselim.medium.com/%C3%BClkeler-baz-al%C4%B1nd%C4%B1%C4%9F%C4%B1nda-temel-i%CC%87%C3%A7me-suyuna-sahip-olman%C4%B1n-%C3%A7ocuklar%C4%B1n-%C3%B6l%C3%BCm%C3%BC-%C3%BCzerine-etkisi-78befdb2f0f8

Github: https://github.com/SelimSavas/SuImkaniveCocukOlumOraniRegresyonAnalizi

## İçindekiler

1.  Verilerin Düzenlenmesi
2.  Normal Dağılımının İncelenmesi
3.  Basit Regresyon Analizi

## 1-Verilerin Düzenlenmesi

### Kütüphaneler

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
from scipy import stats

### Temel İçme Suyu Verisinin Düzenlenmesi

In [ ]:
# basicDrinkingWaterServices.csv -> En azından temel içme suyu hizmetlerini kullanan nüfus (%)

TemizSuİmkanı = pd.read_csv("../input/who-worldhealth-statistics-2020-complete/basicDrinkingWaterServices.csv")
TemizSuİmkanı = TemizSuİmkanı.drop('Indicator',1)
TemizSuİmkanı.columns = ["Lokasyon", "Periyot", "Su_Imkanı"]
TemizSuİmkanı = TemizSuİmkanı[TemizSuİmkanı.Periyot>2016]
TemizSuİmkanı = TemizSuİmkanı[TemizSuİmkanı.Periyot<2018]
TemizSuİmkanı.reset_index(drop=True)

In [ ]:
len(TemizSuİmkanı.Lokasyon.unique()) # 190 Ayrı lokasyonda veri var.

In [ ]:
plt.hist(TemizSuİmkanı.Su_Imkanı) # Su imkanının histogramda dağılımı

### Çocuk Ölümü Oranları Verilerini Düzenleme

In [ ]:
# Çocuk Ölüm Oranları
# neonatalMortalityRate.csv -> Çocukların yaşamın ilk 28 gününde ölme olasılığı.

In [ ]:
ÇocukÖlümOranı = pd.read_csv("../input/who-worldhealth-statistics-2020-complete/neonatalMortalityRate.csv")
ÇocukÖlümOranı = ÇocukÖlümOranı.drop('Indicator',1)
ÇocukÖlümOranı = ÇocukÖlümOranı.drop('Dim1',1)
ÇocukÖlümOranı.columns = ["Lokasyon", "Periyot", "Cocuk_Olum_Oranı"]
ÇocukÖlümOranı = ÇocukÖlümOranı[ÇocukÖlümOranı.Periyot>2016]
ÇocukÖlümOranı = ÇocukÖlümOranı[ÇocukÖlümOranı.Periyot<2018]
ÇocukÖlümOranı.reset_index(drop=True)

In [ ]:
# Bebek Ölüm Oranlarını float hale çevirme

def fonksiyon(ÇocukÖlümOranı):
    return ÇocukÖlümOranı['Cocuk_Olum_Oranı'].split("[")[0]

ÇocukÖlümOranı['Cocuk_Olum_Oranı'] = ÇocukÖlümOranı.apply(fonksiyon, axis=1)
ÇocukÖlümOranı['Cocuk_Olum_Oranı'] = ÇocukÖlümOranı['Cocuk_Olum_Oranı'].astype(float)

### Verileri Birleştirme

In [ ]:
Analiz_1 = pd.merge(ÇocukÖlümOranı,TemizSuİmkanı, on='Lokasyon') # 2017 Yılı Su imkanına ulaşım ve çocuk ölümleri arası korelasyon analizi
Analiz_1 = Analiz_1.drop('Periyot_x',1)
Analiz_1 = Analiz_1.drop('Periyot_y',1)

### Veri Normalize

In [ ]:
Analiz_1.Su_Imkanı = (Analiz_1.Su_Imkanı - Analiz_1.Su_Imkanı.min() )/ (Analiz_1.Su_Imkanı.max() - Analiz_1.Su_Imkanı.min())
Analiz_1.Cocuk_Olum_Oranı = (Analiz_1.Cocuk_Olum_Oranı - Analiz_1.Cocuk_Olum_Oranı.min() )/ (Analiz_1.Cocuk_Olum_Oranı.max() - Analiz_1.Cocuk_Olum_Oranı.min())

### Aykırı değerleri Temizleme

In [ ]:
Analiz_1.Su_Imkanı = Analiz_1.Su_Imkanı[Analiz_1.Su_Imkanı<0.92]
#Analiz_1.Su_Imkanı = Analiz_1.Su_Imkanı[Analiz_1.Su_Imkanı>0]

Analiz_1.Cocuk_Olum_Oranı = Analiz_1.Cocuk_Olum_Oranı[Analiz_1.Cocuk_Olum_Oranı<0.90]
Analiz_1.Cocuk_Olum_Oranı = Analiz_1.Cocuk_Olum_Oranı[Analiz_1.Cocuk_Olum_Oranı>0.04]


Analiz_1.dropna(inplace=True)
Analiz_1.reset_index(drop=True)


In [ ]:
# 45 adet ülkeyi verimizin normal dağılıma sahip olması için analizimizin dışında tuttuk. 
# Çok fazla temel su imkanına sahip olan ülke var olması, verinin dağılımını bozuyordu.

Analiz_1.drop([0,1,2,23,32,34,36,45,47,48,51,66,67,71,74,77,81,84,89,91,93,97,99,101,103,104,109,113,116,119,121,124,130,131,136,137,139,148,154,155,157,159,161,171,176], axis=0, inplace=True)

In [ ]:
Analiz_1.reset_index(drop=True)
Analiz_1.head()

In [ ]:
plt.scatter(Analiz_1.Su_Imkanı.values,Analiz_1.Cocuk_Olum_Oranı.values)
plt.xlabel('x')
plt.ylabel('Y')
plt.title('X y arasındaki ilişki')

## 2- Verinin Normal Dağılımının İncelenmesi

### Betimleyici İstatistik Yöntemleri ile Normal Dağılımı İnceleme



In [ ]:
Analiz_1.describe().T

### Görselleştirme Yöntemleri ile Normal Dağılımı İnceleme

Bins metodu kısacası frekansa göre sayısal verileri gruplara böler. BINS = 9 için histogram oluşturalım ve dağılıma göre en uygun çan eğrisini (bell curve) çizdirerek değerlendirelim.

In [ ]:
# Histogram plot parametreleri
_, bins, _ = plt.hist(Analiz_1.Cocuk_Olum_Oranı, bins = 9,
                      density = 1, alpha = 0.5,
                      ec= 'black' )

# Çan eğrisi için en uygun değerler
mu, sigma = stats.norm.fit(Analiz_1.Cocuk_Olum_Oranı)
best_fit_line = stats.norm.pdf(bins, mu, sigma)

# Çan eğrisi gösterimi
plt.plot(bins, best_fit_line, 'g-o')
plt.xlabel("Çocuk Ölüm Oranları")
plt.ylabel("Frekans")
plt.show()

In [ ]:
# Histogram plot parametreleri
_, bins, _ = plt.hist(Analiz_1.Su_Imkanı, bins = 9,
                      density = 1, alpha = 0.5,
                      ec= 'black' )

# Çan eğrisi için en uygun değerler
mu, sigma = stats.norm.fit(Analiz_1.Su_Imkanı)
best_fit_line = stats.norm.pdf(bins, mu, sigma)

# Çan eğrisi gösterimi
plt.plot(bins, best_fit_line, 'g-o')
plt.xlabel("Su İmkanı Oranı")
plt.ylabel("Frekans")
plt.show()

### Diğer Görselleştirme Yöntemleri ile Nomral Dağılımı İnceleme

In [ ]:
# 6'a 3'lük bir alan için grafik sahası oluşturuyoruz
plt.figure(figsize=(6, 3))

plt.subplot(121)
plt.boxplot(Analiz_1.Su_Imkanı)
plt.title("Su İmkanı")

plt.subplot(122)
plt.boxplot(Analiz_1.Cocuk_Olum_Oranı)
plt.title("Çocuk Ölüm Oranı")


plt.show()

### Kernel Density Estimation (KDE)

In [ ]:
Analiz_1.plot.kde()

### Normallik Varsayım Testlerini Python ile inceleme

In [ ]:
# Skewness (Çarpıklık) ve kurtosis (Basıklık) değerleri

m = Analiz_1.Cocuk_Olum_Oranı
i = Analiz_1.Su_Imkanı

print("Çocuk Ölüm Oranı")
print(f"Skewness: {stats.skew(m)} Kurtosis: {stats.kurtosis(m)}")
print("Su İmkanı Oranı")
print(f"Skewness: {stats.skew(i)} Kurtosis: {stats.kurtosis(i)}")

### Shapiro-Wilk Testi

Normal dağılım sınaması için Shapiro-Wilk ve Kolmogorov-Smirnov en yaygın olarak kullanılan testler olup temelde hipotezleri aynıdır.

* H0: Değişken normal dağılıma sahiptir. p-value > 0.05
* H1: Değişken normal dağılıma sahip değildir. p-value < 0.05

In [ ]:
m = Analiz_1.Cocuk_Olum_Oranı
i = Analiz_1.Su_Imkanı

print("Çocuk Ölüm Oranı")
print(f"T: {stats.shapiro(m)[0]} P-Value: {stats.shapiro(m)[1]}")
print("Su İmkanı Oranı")
print(f"T: {stats.shapiro(i)[0]} P-Value: {stats.shapiro(i)[1]}")

* Her bir değişken için sonuçları incelerken hipotez reddi/kabülu için p-value’ya bakmamız yeterlidir. Buna göre;

* Çocuk Ölüm Oranları'nın ve Su İmkanı Oranları'nın p-value değerlerine baktığımızda 0.05 değerinden büyük olduğu için H0 hipotezini kabul etmiş oluyoruz.

### Kolmogorov-Smirnov
Shapiro-Wilk testi, normalliği belirlemek için popülerdir ve genellikle çok iyi performans gösterir, ancak evrensel olarak en iyisi değildir. Bir değişkeni incelerken tek bir teste bağlı kalmadan sonuçlarınızı doğrulamak için ek yöntemler kullanmanız gerekir. Bunun için Kolmagorov-Smirnov testi uygulanabilir.

In [ ]:
# Kolmogorov–Smirnov test
m = Analiz_1.Cocuk_Olum_Oranı
i = Analiz_1.Su_Imkanı

m_kstest = stats.kstest(m, 'norm',
                         args=(m.mean(),
                               m.std()))
i_kstest = stats.kstest(i, 'norm',
                         args=(i.mean(),
                               i.std()))

print("Çocuk Ölüm Oranı")
print(f"T: {m_kstest[0]} P-Value: {m_kstest[1]}")
print("Su İmkanı Oranı")
print(f"T: {i_kstest[0]} P-Value: {i_kstest[1]}")

In [ ]:
Analiz_1.corr()

## 3- Verinin Basit Regresyon Analizinin Yapılması

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
X = Analiz_1.Su_Imkanı
y = Analiz_1.Cocuk_Olum_Oranı

In [ ]:
plt.scatter(X.values,y.values)
plt.xlabel('Su Imkanı')
plt.ylabel('Çocuk Ölüm Oranı')

In [ ]:
lineer_regresyon = LinearRegression()
lineer_regresyon.fit(X.values.reshape(-1,1),y.values.reshape(-1,1))

In [ ]:
print(lineer_regresyon.intercept_)
print(lineer_regresyon.coef_)

In [ ]:
print("Elde edilen regresyon modeli: Y={}+{}X".format(lineer_regresyon.intercept_,lineer_regresyon.coef_[0]))

### R2 Değeri

In [ ]:
y_predicted = lineer_regresyon.predict(X.values.reshape(-1,1))
r2_score(y,y_predicted)

### Ortalama Mutlak ve Ortalama Karesel Hata Değerleri

In [ ]:
print("Ortalama Mutlak Hata: {} \nOrtalama Karesel Hata: {}".format(
    mean_absolute_error(y, y_predicted), mean_squared_error(y, y_predicted)))

### Regresyon Gösterimi

In [ ]:
random_x = np.array([0, 0.5, 0.99])
plt.scatter(X.values, y.values)
plt.plot(random_x,
         lineer_regresyon.intercept_[0] +
         lineer_regresyon.coef_[0][0] * random_x,
         color='red',
         label='regresyon grafiği')
plt.xlabel('x')
plt.ylabel('Y')
plt.title('X y regresyon analiz')

In [ ]:
g = sns.jointplot(Analiz_1.Su_Imkanı,Analiz_1.Cocuk_Olum_Oranı,
                  size=5,
                  kind="reg", color="r")